In [1]:
# need ipykernel > 6
# ! pip install ipykernel==6.23.1
! pip install git+https://github.com/ginihumer/amumo.git

  Cloning https://github.com/ginihumer/CLIP-explorer.git to /private/var/folders/46/rz1kpyy146n3kk5pyv0pqdh80000gn/T/pip-req-build-rjptox20
  Running command git clone --filter=blob:none --quiet https://github.com/ginihumer/CLIP-explorer.git /private/var/folders/46/rz1kpyy146n3kk5pyv0pqdh80000gn/T/pip-req-build-rjptox20
  Resolved https://github.com/ginihumer/CLIP-explorer.git to commit e5c305fe7ca509355b85110db8f31fb6f301cf56
  Preparing metadata (setup.py) ... done
  Using cached clip-1.0-py3-none-any.whl
  Using cached ipywidgets-8.0.6-py3-none-any.whl (138 kB)
  Using cached widgetsnbextension-4.0.7-py3-none-any.whl (2.1 MB)
  Created wheel for CLIP-explorer: filename=CLIP_explorer-0.1.30-py3-none-any.whl size=1428968 sha256=04bfaa7ebee9a3c1b648da53d8bf77c3b901e7f9e57b15d9eec17c5f86cbed6a
  Stored in directory: /private/var/folders/46/rz1kpyy146n3kk5pyv0pqdh80000gn/T/pip-ephem-wheel-cache-s42frk7o/wheels/09/fc/8b/3b8a1f10143be547929a0768142222f15b2860f13226a18139
Successfully built

In [2]:
from amumo import model
from amumo import data

In [3]:
dataset = data.DiffusionDB_Dataset(path="2m_first_1k")
def get_data_helper(dataset, filters=[], method=any):
    all_images, all_prompts = dataset.get_filtered_data(filters, method=method)
    print(len(all_images))

    dataset_name = dataset.name
    if len(filters) > 0:
        dataset_name = dataset_name + '_filter-' + method.__name__ + '_' + '-'.join(filters)
    else:
        dataset_name = dataset_name + '_size-%i'%len(all_images)

    return all_images, all_prompts, dataset_name

all_images, all_prompts, dataset_name = get_data_helper(dataset, filters=[], method=any) # filters = ["dog"], method=all
dataset_name

Found cached dataset diffusiondb (/Users/christina/.cache/huggingface/datasets/poloclub___diffusiondb/2m_first_1k/0.9.1/b3bc1e64570dc7149af62c4bac49ecfbce16b683dd4fee083292fae1afa95f7c)


  0%|          | 0/1 [00:00<?, ?it/s]

100


'DiffusionDB_size-100'

In [ ]:
class CustomModel(model.CLIPModelInterface):
    available_models = clip.available_models()
    model_name = 'CLIP'

    def __init__(self, name='RN50', device='cpu') -> None:
        super().__init__(name, device)
        self.model, self.preprocess = clip.load(name, device=device)
        self.model.eval()
        self.logit_scale = self.model.logit_scale

    def encode_image(self, images):
        images = [self.preprocess(i) for i in images]
        image_input = torch.tensor(np.stack(images)).to(self.device)
        return self.model.encode_image(image_input).float().cpu()

    def encode_text(self, texts):
        text_tokens = clip.tokenize(texts, truncate = True).to(self.device)
        return self.model.encode_text(text_tokens).float().cpu()

In [4]:
from amumo.widgets import CLIPExplorerWidget


clipexplorer_widget = CLIPExplorerWidget(dataset_name, all_images, all_prompts, models=[CustomModel()])
clipexplorer_widget

/Users/christina/anaconda3/envs/myenv/lib/python3.9/site-packages/umap/distances.py:1063: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @numba.jit()
/Users/christina/anaconda3/envs/myenv/lib/python3.9/site-packages/umap/distances.py:1071: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @numba.jit()
/Users/christina/anaconda3/envs/myenv/lib/python3.9/site-packages/

Loading model from /Users/christina/anaconda3/envs/myenv/lib/python3.9/site-packages/clipexplorer/CLOOB_local/training/model_configs/RN50.json


/Users/christina/anaconda3/envs/myenv/lib/python3.9/site-packages/clipexplorer/widgets.py:356: FutureWarning: The input object of type 'PngImageFile' is an array-like implementing one of the corresponding protocols (`__array__`, `__array_interface__` or `__array_struct__`); but not a sequence (or 0-D). In the future, this object will be coerced as if it was first converted using `np.array(obj)`. To retain the old behaviour, you have to either modify the type 'PngImageFile', or assign to an empty array created with `np.empty(correct_shape, dtype=object)`.
  self.all_images = np.array(all_images)
/Users/christina/anaconda3/envs/myenv/lib/python3.9/site-packages/clipexplorer/widgets.py:356: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  self.all_images = np.array(all_imag

found cached embeddings for DiffusionDB_size-100_CLIP_RN50


CLIPExplorerWidget(children=(VBox(children=(HBox(children=(Dropdown(description='Model: ', options=('CLIP', 'O…